# RAGAS 기반 리랭커 성능 비교 분석

이 노트북은 다양한 리랭커들의 성능을 RAGAS 메트릭을 사용하여 비교 분석합니다.

## 평가 대상 리랭커들
- BM25 계열 (5개)
- CrossEncoder 계열 (7개)
- BGE 계열 (4개)
- Embedding 계열 (5개)
- Hybrid 계열 (4개)
- LLM 계열 (3개)
- Rules 계열 (1개)

## RAGAS 메트릭
- **Context Precision**: 검색된 컨텍스트의 정확성
- **Context Recall**: 관련 컨텍스트의 검색 완성도
- **Faithfulness**: 생성된 답변의 신뢰성
- **Answer Relevancy**: 답변의 관련성


In [ ]:
# 환경 설정 및 패키지 설치
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import List, Dict, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 프로젝트 경로 설정
BASE_DIR = Path.cwd()
RERANKERS_DIR = BASE_DIR / "RAG_with_Various_Rerankers"
EMB_PATH = BASE_DIR / "output_chunks_with_embeddings.json"
QA_PATH = BASE_DIR / "real_estate_tax_QA.json"

# 패키지 경로 추가
sys.path.insert(0, str(RERANKERS_DIR))

print(f"BASE_DIR: {BASE_DIR}")
print(f"RERANKERS_DIR: {RERANKERS_DIR}")
print(f"EMB_PATH exists: {EMB_PATH.exists()}")
print(f"QA_PATH exists: {QA_PATH.exists()}")


In [ ]:
# 필요한 패키지 설치 및 import
try:
    import ragas
    from ragas import evaluate
    from ragas.metrics import (
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy
    )
    print("✅ RAGAS 패키지가 이미 설치되어 있습니다.")
except ImportError:
    print("📦 RAGAS 패키지를 설치합니다...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ragas[all]"])


In [ ]:
# 정답 데이터 로드 및 분석
with open(QA_PATH, 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

print(f"총 질문-답변 쌍: {len(qa_data)}개")
print(f"첫 번째 샘플:")
print(f"질문: {qa_data[0]['question']}")
print(f"정답: {qa_data[0]['ground_truth']}")
print(f"컨텍스트 수: {len(qa_data[0]['ground_truth_contexts'])}")
print(f"토픽: {qa_data[0]['metadata']['topic']}")

# 토픽별 분포 분석
topic_counts = {}
for item in qa_data:
    topic = item['metadata']['topic']
    topic_counts[topic] = topic_counts.get(topic, 0) + 1

print("\n토픽별 분포:")
for topic, count in sorted(topic_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {topic}: {count}개")


In [ ]:
# 평가용 샘플 데이터 준비 (전체 데이터가 많으므로 샘플링)
import random

# 재현 가능한 랜덤 샘플링
random.seed(42)
np.random.seed(42)

# 토픽별로 균등하게 샘플링 (각 토픽당 최대 3개)
sampled_data = []
topic_samples = {}

for item in qa_data:
    topic = item['metadata']['topic']
    if topic_samples.get(topic, 0) < 3:
        sampled_data.append(item)
        topic_samples[topic] = topic_samples.get(topic, 0) + 1

# 최대 30개 샘플로 제한
if len(sampled_data) > 30:
    sampled_data = random.sample(sampled_data, 30)

print(f"평가용 샘플 데이터: {len(sampled_data)}개")
print("샘플 토픽 분포:")
sample_topic_counts = {}
for item in sampled_data:
    topic = item['metadata']['topic']
    sample_topic_counts[topic] = sample_topic_counts.get(topic, 0) + 1

for topic, count in sorted(sample_topic_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {topic}: {count}개")


In [ ]:
# 리랭커 모듈 목록 정의
RERANKER_MODULES = [
    # BM25 리랭커들 (5개)
    ("BM25_Reranker.RAG_BM25_Rerank_FINAL", "BM25 기본", "LegalRAGSystemBM25Rerank"),
    ("BM25_Reranker.RAG_BM25_CharNgram_Rerank_FINAL", "BM25 CharNgram", "LegalRAGSystemBM25CharNgram"),
    ("BM25_Reranker.RAG_BM25_Kiwi_Rerank_FINAL", "BM25 Kiwi", "LegalRAGSystemBM25Kiwi"),
    ("BM25_Reranker.RAG_BM25_Regex_Rerank_FINAL", "BM25 Regex", "LegalRAGSystemBM25Regex"),
    ("BM25_Reranker.RAG_BM25_Stopword_Rerank_FINAL", "BM25 Stopword", "LegalRAGSystemBM25Stopword"),
    
    # CrossEncoder 리랭커들 (7개)
    ("CrossEncoder.RAG_CE_MiniLM_L6_Rerank_FINAL", "CrossEncoder MiniLM L6", "LegalRAGSystemMiniLML6"),
    ("CrossEncoder.RAG_CE_MiniLM_L12_Rerank_FINAL", "CrossEncoder MiniLM L12", "LegalRAGSystemMiniLML12"),
    ("CrossEncoder.RAG_CE_Electra_Rerank_FINAL", "CrossEncoder Electra", "LegalRAGSystemElectra"),
    ("CrossEncoder.RAG_CE_E5_Mistral_Rerank_FINAL", "CrossEncoder E5 Mistral", "LegalRAGSystemE5Mistral"),
    ("CrossEncoder.RAG_Cohere_Rerank_FINAL", "Cohere", "LegalRAGSystemCohere"),
    ("CrossEncoder.RAG_MXBAI_Rerank_FINAL", "MXBAI", "LegalRAGSystemMXBAI"),
    
    # BGE 계열 (4개)
    ("CrossEncoder.BGE 계열.RAG_BGE_Base_Rerank_FINAL", "BGE Base", "LegalRAGSystemBGEBase"),
    ("CrossEncoder.BGE 계열.RAG_BGE_Large_Rerank_FINAL", "BGE Large", "LegalRAGSystemBGELarge"),
    ("CrossEncoder.BGE 계열.RAG_BGE_v2m3_Rerank_FINAL", "BGE v2 M3", "LegalRAGSystemBGEv2m3"),
    ("CrossEncoder.BGE 계열.RAG_Flashrank_BGEv2m3_Rerank_FINAL", "Flashrank BGE v2m3", "LegalRAGSystemFlashrankBGEv2m3"),
    
    # Embedding 리랭커들 (5개)
    ("Embedding_Reranker.RAG_EmbeddingCosine_E5_Rerank_FINAL", "Embedding E5", "LegalRAGSystemEmbeddingE5"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_GTE_Rerank_FINAL", "Embedding GTE", "LegalRAGSystemEmbeddingGTE"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_MPNet_Rerank_FINAL", "Embedding MPNet", "LegalRAGSystemEmbeddingMPNet"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_Paraphrase_Rerank_FINAL", "Embedding Paraphrase", "LegalRAGSystemEmbeddingParaphrase"),
    ("Embedding_Reranker.RAG_EmbeddingCosine_Stella_Rerank_FINAL", "Embedding Stella", "LegalRAGSystemEmbeddingStella"),
    
    # Hybrid 리랭커들 (4개)
    ("Hybrid_Reranker.RAG_CombSum_Rerank_FINAL", "Hybrid CombSum", "LegalRAGSystemCombSum"),
    ("Hybrid_Reranker.RAG_CombMNZ_Rerank_FINAL", "Hybrid CombMNZ", "LegalRAGSystemCombMNZ"),
    ("Hybrid_Reranker.RAG_RRF_Rerank_FINAL", "Hybrid RRF", "LegalRAGSystemRRF"),
    ("Hybrid_Reranker.RAG_WeightedSum_BM25_Embed_Rerank_FINAL", "Hybrid WeightedSum", "LegalRAGSystemWeightedSum"),
    
    # LLM 리랭커들 (3개)
    ("LLM_Reranker.RAG_LLM_Rerank_FINAL", "LLM 기본", "LegalRAGSystemLLM"),
    ("LLM_Reranker.RAG_LLM_Listwise_Rerank_FINAL", "LLM Listwise", "LegalRAGSystemLLMListwise"),
    ("LLM_Reranker.RAG_LLM_Pairwise_Rerank_FINAL", "LLM Pairwise", "LegalRAGSystemLLMPairwise"),
    
    # Rules 리랭커들 (1개)
    ("Rules_Reranker.RAG_LegalRuleBoost_Rerank_FINAL", "Legal Rule Boost", "LegalRAGSystemLegalRuleBoost")
]

print(f"총 {len(RERANKER_MODULES)}개의 리랭커 모듈을 평가합니다.")


In [ ]:
# 리랭커 로드 및 초기화 함수
def load_reranker(module_path: str, class_name: str) -> Any:
    """리랭커 모듈을 동적으로 로드합니다."""
    try:
        module = __import__(module_path, fromlist=[class_name])
        reranker_class = getattr(module, class_name)
        return reranker_class(embeddings_file=str(EMB_PATH))
    except Exception as e:
        print(f"❌ {module_path} 로드 실패: {e}")
        return None

# 테스트용 리랭커 로드 (처음 몇 개만)
test_rerankers = {}
for module_path, name, class_name in RERANKER_MODULES[:3]:  # 처음 3개만 테스트
    print(f"로딩 중: {name}...")
    reranker = load_reranker(module_path, class_name)
    if reranker:
        test_rerankers[name] = reranker
        print(f"✅ {name} 로드 성공")
    else:
        print(f"❌ {name} 로드 실패")

print(f"\n성공적으로 로드된 리랭커: {len(test_rerankers)}개")


In [ ]:
# RAGAS 평가를 위한 데이터 준비 함수
def prepare_ragas_data(sampled_data: List[Dict], reranker: Any, reranker_name: str) -> List[Dict]:
    """RAGAS 평가를 위한 데이터를 준비합니다."""
    ragas_data = []
    
    for i, item in enumerate(sampled_data):
        try:
            question = item['question']
            ground_truth = item['ground_truth']
            ground_truth_contexts = item['ground_truth_contexts']
            
            # 리랭커로 문서 검색
            docs = reranker.retriever.invoke(question)
            
            # RAGAS 형식으로 변환
            ragas_item = {
                'question': question,
                'answer': ground_truth,  # 정답을 답변으로 사용
                'contexts': [doc.page_content for doc in docs],
                'ground_truth': ground_truth,
                'ground_truths': ground_truth_contexts
            }
            ragas_data.append(ragas_item)
            
        except Exception as e:
            print(f"❌ {reranker_name} - 질문 {i+1} 처리 실패: {e}")
            continue
    
    return ragas_data

# 테스트 데이터 준비
print("RAGAS 평가 데이터 준비 중...")
test_ragas_data = {}

for name, reranker in test_rerankers.items():
    print(f"\n{name} 데이터 준비 중...")
    ragas_data = prepare_ragas_data(sampled_data, reranker, name)
    test_ragas_data[name] = ragas_data
    print(f"✅ {name}: {len(ragas_data)}개 샘플 준비 완료")

print(f"\n총 {len(test_ragas_data)}개 리랭커의 RAGAS 데이터 준비 완료")


In [ ]:
# RAGAS 평가 실행 함수
def evaluate_with_ragas(ragas_data: List[Dict], reranker_name: str) -> Dict[str, float]:
    """RAGAS 메트릭으로 리랭커를 평가합니다."""
    try:
        # DataFrame으로 변환
        df = pd.DataFrame(ragas_data)
        
        # RAGAS 평가 실행
        metrics = [
            context_precision,
            context_recall,
            faithfulness,
            answer_relevancy
        ]
        
        result = evaluate(df, metrics=metrics)
        
        # 결과 추출
        scores = {
            'context_precision': float(result['context_precision']),
            'context_recall': float(result['context_recall']),
            'faithfulness': float(result['faithfulness']),
            'answer_relevancy': float(result['answer_relevancy'])
        }
        
        print(f"✅ {reranker_name} 평가 완료")
        for metric, score in scores.items():
            print(f"  {metric}: {score:.4f}")
        
        return scores
        
    except Exception as e:
        print(f"❌ {reranker_name} 평가 실패: {e}")
        return {
            'context_precision': 0.0,
            'context_recall': 0.0,
            'faithfulness': 0.0,
            'answer_relevancy': 0.0
        }

# 테스트 리랭커들 평가
print("RAGAS 평가 시작...")
evaluation_results = {}

for name, ragas_data in test_ragas_data.items():
    print(f"\n{name} 평가 중...")
    scores = evaluate_with_ragas(ragas_data, name)
    evaluation_results[name] = scores

print("\n🎉 평가 완료!")
print(f"평가된 리랭커 수: {len(evaluation_results)}개")


In [ ]:
# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(evaluation_results).T
results_df = results_df.round(4)

# 종합 점수 계산 (평균)
results_df['overall_score'] = results_df.mean(axis=1)
results_df = results_df.sort_values('overall_score', ascending=False)

print("📊 RAGAS 평가 결과 (상위 10개)")
print("=" * 80)
print(results_df.head(10))

# 결과 저장
results_df.to_csv('ragas_evaluation_results.csv', encoding='utf-8-sig')
print("\n💾 결과가 'ragas_evaluation_results.csv'에 저장되었습니다.")


In [ ]:
# 시각화 1: 메트릭별 성능 비교
plt.figure(figsize=(15, 10))

# 서브플롯 설정
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('RAGAS 메트릭별 리랭커 성능 비교', fontsize=16, fontweight='bold')

metrics = ['context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']
metric_names = ['Context Precision', 'Context Recall', 'Faithfulness', 'Answer Relevancy']

for i, (metric, name) in enumerate(zip(metrics, metric_names)):
    ax = axes[i//2, i%2]
    
    # 데이터 준비
    data = results_df[metric].sort_values(ascending=True)
    
    # 막대 그래프
    bars = ax.barh(range(len(data)), data.values, color='skyblue', alpha=0.7)
    
    # 레이블 설정
    ax.set_yticks(range(len(data)))
    ax.set_yticklabels(data.index, fontsize=8)
    ax.set_xlabel(name, fontsize=10)
    ax.set_title(f'{name} 비교', fontsize=12, fontweight='bold')
    
    # 값 표시
    for j, bar in enumerate(bars):
        width = bar.get_width()
        ax.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                f'{width:.3f}', ha='left', va='center', fontsize=8)
    
    # 그리드 추가
    ax.grid(axis='x', alpha=0.3)
    ax.set_xlim(0, 1.0)

plt.tight_layout()
plt.savefig('ragas_metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 시각화 2: 종합 성능 순위
plt.figure(figsize=(12, 8))

# 종합 점수 기준 정렬
overall_scores = results_df['overall_score'].sort_values(ascending=True)

# 막대 그래프
bars = plt.barh(range(len(overall_scores)), overall_scores.values, 
                color='lightcoral', alpha=0.8, edgecolor='darkred')

# 레이블 설정
plt.yticks(range(len(overall_scores)), overall_scores.index)
plt.xlabel('종합 점수 (Overall Score)', fontsize=12)
plt.title('리랭커 종합 성능 순위 (RAGAS 기준)', fontsize=14, fontweight='bold')

# 값 표시
for i, bar in enumerate(bars):
    width = bar.get_width()
    plt.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{width:.3f}', ha='left', va='center', fontsize=10)

# 그리드 추가
plt.grid(axis='x', alpha=0.3)
plt.xlim(0, 1.0)

plt.tight_layout()
plt.savefig('reranker_overall_ranking.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n🏆 상위 5개 리랭커:")
for i, (name, score) in enumerate(overall_scores.tail(5).items()):
    print(f"{i+1}. {name}: {score:.4f}")


In [ ]:
# 시각화 3: 히트맵 - 메트릭별 상세 성능
plt.figure(figsize=(12, 8))

# 히트맵 데이터 준비
heatmap_data = results_df[['context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']]

# 히트맵 생성
sns.heatmap(heatmap_data, 
            annot=True, 
            cmap='YlOrRd', 
            fmt='.3f',
            cbar_kws={'label': '점수'},
            annot_kws={'size': 8})

plt.title('리랭커별 RAGAS 메트릭 히트맵', fontsize=14, fontweight='bold')
plt.xlabel('RAGAS 메트릭', fontsize=12)
plt.ylabel('리랭커', fontsize=12)
plt.xticks(rotation=45)
plt.yticks(rotation=0)

plt.tight_layout()
plt.savefig('ragas_metrics_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# 상세 분석 및 권장사항
print("🔍 상세 분석 및 권장사항")
print("=" * 60)

# 최고 성능 리랭커
best_reranker = results_df.index[0]
best_score = results_df.iloc[0]['overall_score']
print(f"\n🏆 최고 성능 리랭커: {best_reranker}")
print(f"   종합 점수: {best_score:.4f}")

# 메트릭별 최고 성능
print("\n📊 메트릭별 최고 성능:")
for metric in ['context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']:
    best_idx = results_df[metric].idxmax()
    best_value = results_df[metric].max()
    print(f"   {metric}: {best_idx} ({best_value:.4f})")

# 성능 분석
print("\n📈 성능 분석:")
avg_scores = results_df.mean()
print(f"   전체 평균 종합 점수: {avg_scores['overall_score']:.4f}")
print(f"   Context Precision 평균: {avg_scores['context_precision']:.4f}")
print(f"   Context Recall 평균: {avg_scores['context_recall']:.4f}")
print(f"   Faithfulness 평균: {avg_scores['faithfulness']:.4f}")
print(f"   Answer Relevancy 평균: {avg_scores['answer_relevancy']:.4f}")

# 권장사항
print("\n💡 권장사항:")
if best_score > 0.8:
    print("   ✅ 우수한 성능을 보이는 리랭커가 있습니다.")
else:
    print("   ⚠️  전체적인 성능 개선이 필요합니다.")

if avg_scores['context_precision'] < 0.7:
    print("   📝 Context Precision 개선이 필요합니다.")
if avg_scores['context_recall'] < 0.7:
    print("   📝 Context Recall 개선이 필요합니다.")
if avg_scores['faithfulness'] < 0.7:
    print("   📝 Faithfulness 개선이 필요합니다.")
if avg_scores['answer_relevancy'] < 0.7:
    print("   📝 Answer Relevancy 개선이 필요합니다.")


In [ ]:
# 전체 리랭커 평가 실행 (선택사항)
print("🚀 전체 리랭커 평가를 실행하시겠습니까?")
print("⚠️  주의: 이 작업은 시간이 오래 걸릴 수 있습니다.")
print("실행하려면 아래 셀의 주석을 해제하고 실행하세요.")

# 전체 평가 코드 (주석 처리)
"""
# 전체 리랭커 로드 및 평가
print("전체 리랭커 로딩 시작...")
all_rerankers = {}

for module_path, name, class_name in RERANKER_MODULES:
    print(f"로딩 중: {name}...")
    reranker = load_reranker(module_path, class_name)
    if reranker:
        all_rerankers[name] = reranker
        print(f"✅ {name} 로드 성공")
    else:
        print(f"❌ {name} 로드 실패")

print(f"\n전체 로드된 리랭커: {len(all_rerankers)}개")

# 전체 RAGAS 데이터 준비
print("\n전체 RAGAS 데이터 준비 중...")
all_ragas_data = {}

for name, reranker in all_rerankers.items():
    print(f"\n{name} 데이터 준비 중...")
    ragas_data = prepare_ragas_data(sampled_data, reranker, name)
    all_ragas_data[name] = ragas_data
    print(f"✅ {name}: {len(ragas_data)}개 샘플 준비 완료")

# 전체 평가 실행
print("\n전체 RAGAS 평가 시작...")
all_evaluation_results = {}

for name, ragas_data in all_ragas_data.items():
    print(f"\n{name} 평가 중...")
    scores = evaluate_with_ragas(ragas_data, name)
    all_evaluation_results[name] = scores

print("\n🎉 전체 평가 완료!")

# 전체 결과 저장
all_results_df = pd.DataFrame(all_evaluation_results).T
all_results_df['overall_score'] = all_results_df.mean(axis=1)
all_results_df = all_results_df.sort_values('overall_score', ascending=False)
all_results_df.to_csv('all_ragas_evaluation_results.csv', encoding='utf-8-sig')
print("전체 결과가 'all_ragas_evaluation_results.csv'에 저장되었습니다.")
"""
